In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import zipfile
import io
import seaborn as sns
import matplotlib.pyplot as plt


# fetch the dataset usin the Kaggle Api

#dataset = "anandshaw2001/netflix-movies-and-tv-shows"
#kaggle.api.dataset_download_files(dataset,path='/Users/qurbanovem/Documents' ,unzip=True)

# Load the dataset directly into Pandas 

netflix_df=pd.read_csv("/kaggle/input/netflix-movies-and-tv-shows/netflix_titles.csv")

netflix_df.head(10)

In [ ]:
#explation dataset




print('Data information')
print(netflix_df.info())

print('Data describe')
print(netflix_df.describe())

print('Null count for each column')
print(netflix_df.isnull().sum())


In [ ]:
# cheking unique value count per column

print('Unique Values for each column')

for clmn in netflix_df.columns:
    print(f"\n{clmn} : {netflix_df[clmn].nunique()} unique values ")

In [ ]:
# Selecting top 10 country and replacing NA values 

country=netflix_df['country'].str.split(', ').explode().reset_index(drop=True)
top_countries=country.value_counts().head(10)
countries=country.value_counts().head(10).index.tolist()


def new_column(row):

    if pd.isna(row):
        return 'Not exsist'

    country_list=row.split(', ') if isinstance(row,str) else []
    
    for cntr in country_list:
        if cntr in countries:
            return cntr
    return 'Others'

netflix_df['selected_coutnry']=netflix_df['country'].apply(new_column)
netflix_df.head(10)

In [ ]:
import seaborn as sns
import  matplotlib.pyplot  as plt

selected_countries=netflix_df['selected_coutnry'].value_counts()
plt.figure(figsize=(12,6))
selected_countries.plot(kind='bar')
plt.title ('Countries Count')
plt.xlabel('Countries')
plt.ylabel('Count of Content')
plt.show()

In [ ]:
top_5_content = netflix_df['listed_in'].str.split(', ').explode().value_counts().head(5).index.tolist()

# Step 2: Define the Combined Function
def new_column_combined(row):
    if pd.isna(row):
        return 'Not Exist'

    content_list = row.split(', ') if isinstance(row, str) else []

    # First, check for a 'TV' category
    for cont in content_list:
        if 'TV' in cont:
            selected_content = cont
            break
    else:
        # If no 'TV' category, take the first value
        selected_content = content_list[0] if content_list else 'Not Exist'

    # Check if selected content is in the top 5
    return selected_content if selected_content in top_5_content else 'Others_content'

In [ ]:
netflix_df['content_top_5'] = netflix_df['listed_in'].apply(new_column_combined)
netflix_df.head(10)


In [ ]:
# draw histogram to identfy count of top 15 content category according to type 

plt.figure(figsize=(10,5))
sns.countplot(data=netflix_df , x='content_top_5' , hue='type' , palette='Set2')
plt.xlabel('Content_Category')
plt.ylabel('Count')
plt.title('Count of Content by type')
plt.xticks(rotation=90)
plt.legend(title='Type')
plt.show()


In [ ]:
# create new column according to duration column
netflix_df['duration_num']=netflix_df['duration'].str.extract('(\d+)').astype(float)
netflix_df['duration_type'] = netflix_df['duration_num'].apply(lambda x: 'Whole_part' if x > 20 else 'Session_part')
netflix_df.head(10)

In [ ]:
# chek distrubition of duration type columun

plt.figure(figsize=(10,5))
sns.histplot(data=netflix_df ,x='duration_type' , hue='type' , multiple='dodge' , shrink=0.8 ,palette='Set1')
plt.xlabel('duration_type')
plt.ylabel('count')
plt.title('Duration type distribution by type')
plt.legend(title='Type')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# select top 5 director 


#directors=netflix_df['director'].dropna().str.split(', ').explode().unique()
top_5_director=netflix_df['director'].dropna().str.split(', ').explode().value_counts().head(5).index.tolist()
top_5_director


def top_5_directors(row):
    directors=row.split(', ') if isinstance(row,str) else []

    for drctr in directors:
        if drctr in top_5_director:
            return drctr
    return 'other director'



netflix_df['top_5_directors']=netflix_df['director'].apply(top_5_directors)

In [ ]:
# Demonstrated of Distribution of Top 5 directors by Type

plt.figure(figsize=(10,5))
sns.stripplot(x=netflix_df['type'] , y=netflix_df['top_5_directors'] ,jitter=True ,size=8 )


plt.xlabel('Count')
plt.ylabel('Director')
plt.title('Distribution of Directors by Type (Movies vs. Shows)')

plt.show()

In [ ]:
# removing spaces
netflix_df['date_added']=netflix_df['date_added'].str.strip()

# convert columun to datetime
netflix_df['date_added']=pd.to_datetime(netflix_df['date_added'],format='mixed')

#remove rows contains null value in data_aded columun
netflix_df=netflix_df[~netflix_df['date_added'].isnull()]

netflix_df['date_period']=netflix_df['date_added'].dt.to_period('M')

netflix_df['Month']=netflix_df['date_added'].dt.month

netflix_df['Year']=netflix_df['date_added'].dt.year


print(netflix_df.count())
print(netflix_df['date_added'].dtypes)


In [ ]:
netflix_df.columns

In [ ]:
# Demonstrate content count by Year 

#netflix_df.groupby('date_period')['date_period'].size().sort_values(ascending=False).head(10)
#netflix_df['Year']=netflix_df['date_added'].dt.year
netflix_df
count_by_year=netflix_df.groupby('Year')['Year'].count()
plt.figure(figsize=(10,5))
plt.plot(count_by_year.index ,count_by_year.values ,marker='o' ,linestyle='-' ,color='b' )
plt.xlabel('Year')
plt.ylabel ('Count')
plt.title ('Count of contents by Year')
plt.grid(True)
plt.show()

In [ ]:
# remove and modify  rows where  rating and duration columuns are null 
#modify show_id extract only integr values
netflix_df=netflix_df[(netflix_df['rating'].notna())& (netflix_df['duration'].notna()) ]
netflix_df['show_id']=netflix_df['show_id'].str.extract('(\d+)').astype(int)
netflix_df.info()
netflix_df.head(10)

netflix_df[netflix_df['listed_in'].str.contains('TV',case=False,na=False)].groupby('type')['listed_in'].count()

In [ ]:
dummies_columns=dummies_columns=['show_id','duration_type','content_top_5','top_5_directors']
#dummies_df
dummies_df=netflix_df[dummies_columns]
#dummies_df
netflix_dummies=pd.get_dummies(dummies_df ,columns=['duration_type','content_top_5','top_5_directors']).astype(int)
netflix_dummies.head(10)

In [ ]:
df_merged=netflix_df.merge(netflix_dummies , on=['show_id'],how='inner')
print(df_merged['type'].value_counts())
df_merged['target']=np.where(df_merged['type']=='Movie',0,1)

print(f"target_values_propartion : {df_merged['target'].value_counts(normalize=True)}")
print(df_merged.columns)

In [ ]:
# create train and OOT (Validation dataset)

column_list=['show_id','release_year','duration_num','Month', 'Year','duration_type_Session_part',
       'duration_type_Whole_part', 'content_top_5_Documentaries',
       'content_top_5_Dramas', 'content_top_5_International Movies',
       'content_top_5_International TV Shows', 'content_top_5_Others_content','target'
       ]

df=df_merged[column_list]
df.head(10)

train_dataset=df[~df['Year'].isin([2020])]
print(f"train dataset count : {train_dataset.shape}")
OOT=df[df['Year'].isin([2020])]
print(f"train dataset count : {OOT.shape}")

print(f" Validation dataset target value counts: {OOT['target'].value_counts()}")
#df.head(10)


In [ ]:
# train test split in order fit model on train dataset

from sklearn.model_selection import train_test_split

X=train_dataset.drop(columns=['target'])
y=train_dataset['target']

X_train ,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2 ,random_state=42)


print(f"Number of rows in X_train : {X_train.shape}")
print(f"Number of rows in X_test : {X_test.shape}")
print(f"Number of rows in y_train : {y_train.shape}")
print(f"Number of rows in y_test : {y_test.shape}")

In [ ]:
X_train.info()

In [ ]:
# Fitting Model by using Random Forest Alghoritim
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report



# Initialize Random Forest Classifier
rf_model = RandomForestClassifier(max_depth=10,n_estimators=50, random_state=42)

# Fit the model to training data
rf_model.fit(X_train, y_train)

In [ ]:
# Predict on test data
y_pred = rf_model.predict(X_test)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Evaluate Model

from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay

roc_auc = roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1])  
print(f"ROC AUC Score: {roc_auc:.4f}")

In [ ]:
cm = confusion_matrix(y_test, y_pred)

# Display as a heatmap
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0,1], yticklabels=[0,1])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
X_val=OOT.drop(columns=['target'])
y_val=OOT['target']

print(f"Number of rows in X_val : {X_val.shape}")
print(f"Number of rows in y_val : {y_val.shape}")

roc_auc = roc_auc_score(y_val, rf_model.predict_proba(X_val)[:, 1])  
roc_auc


In [ ]:
#feature Importance
feature_importance=rf_model.feature_importances_
features=X.columns

importance_df = pd.DataFrame({"Feature": features, "Importance": feature_importance})
importance_df = importance_df.sort_values(by="Importance", ascending=False)

#Plot using Seaborn
plt.figure(figsize=(12, 6))
sns.barplot(x="Importance", y="Feature", data=importance_df, palette="viridis")

plt.title("Feature Importance")
plt.xlabel("Importance Score")
plt.ylabel("Feature Name")
plt.show()

In [ ]:
importance_df